# Google drive mount

In [1]:
# 계정 드라이브와 마운트하기
# 모두 선택해서 동의해야 마운트가 오류없이 됩니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 파일 가져오기

In [ ]:
!ls /content/drive/MyDrive/Kaggle02

rokey-boot-camp-mini-competition.zip


In [ ]:
!unzip /content/drive/MyDrive/***/rokey-boot-camp-2nd-competition.zip -d data

Streaming output truncated to the last 5000 lines.
  inflating: data/train_data/00000.png  
  inflating: data/train_data/00001.png  
  inflating: data/train_data/00002.png  
  inflating: data/train_data/00003.png  
  inflating: data/train_data/00004.png  
  inflating: data/train_data/00005.png  
  inflating: data/train_data/00006.png  
  inflating: data/train_data/00007.png  
  inflating: data/train_data/00008.png  
  inflating: data/train_data/00009.png  
  inflating: data/train_data/00010.png  
  inflating: data/train_data/00011.png  
  inflating: data/train_data/00012.png  
  inflating: data/train_data/00013.png  
  inflating: data/train_data/00014.png  
  inflating: data/train_data/00015.png  
  inflating: data/train_data/00016.png  
  inflating: data/train_data/00017.png  
  inflating: data/train_data/00018.png  
  inflating: data/train_data/00019.png  
  inflating: data/train_data/00020.png  
  inflating: data/train_data/00021.png  
  inflating: data/train_data/00022.png  
  infl

# 파일 구조 확인

In [ ]:
!ls data/Kaggle02

test_data  train_data  train_data.csv


In [ ]:
!ls data/Kaggle02/train

00000.png  00715.png  01430.png  02145.png  02860.png  03575.png  04290.png
00001.png  00716.png  01431.png  02146.png  02861.png  03576.png  04291.png
00002.png  00717.png  01432.png  02147.png  02862.png  03577.png  04292.png
00003.png  00718.png  01433.png  02148.png  02863.png  03578.png  04293.png
00004.png  00719.png  01434.png  02149.png  02864.png  03579.png  04294.png
00005.png  00720.png  01435.png  02150.png  02865.png  03580.png  04295.png
00006.png  00721.png  01436.png  02151.png  02866.png  03581.png  04296.png
00007.png  00722.png  01437.png  02152.png  02867.png  03582.png  04297.png
00008.png  00723.png  01438.png  02153.png  02868.png  03583.png  04298.png
00009.png  00724.png  01439.png  02154.png  02869.png  03584.png  04299.png
00010.png  00725.png  01440.png  02155.png  02870.png  03585.png  04300.png
00011.png  00726.png  01441.png  02156.png  02871.png  03586.png  04301.png
00012.png  00727.png  01442.png  02157.png  02872.png  03587.png  04302.png
00013.png  0

In [ ]:
!ls data/Kaggle02/test/

00000.png  01143.png  02286.png  03429.png  04572.png  05715.png  06858.png
00001.png  01144.png  02287.png  03430.png  04573.png  05716.png  06859.png
00002.png  01145.png  02288.png  03431.png  04574.png  05717.png  06860.png
00003.png  01146.png  02289.png  03432.png  04575.png  05718.png  06861.png
00004.png  01147.png  02290.png  03433.png  04576.png  05719.png  06862.png
00005.png  01148.png  02291.png  03434.png  04577.png  05720.png  06863.png
00006.png  01149.png  02292.png  03435.png  04578.png  05721.png  06864.png
00007.png  01150.png  02293.png  03436.png  04579.png  05722.png  06865.png
00008.png  01151.png  02294.png  03437.png  04580.png  05723.png  06866.png
00009.png  01152.png  02295.png  03438.png  04581.png  05724.png  06867.png
00010.png  01153.png  02296.png  03439.png  04582.png  05725.png  06868.png
00011.png  01154.png  02297.png  03440.png  04583.png  05726.png  06869.png
00012.png  01155.png  02298.png  03441.png  04584.png  05727.png  06870.png
00013.png  0

In [ ]:
import json

data = json.load(open("data/Kaggle02/class_name.json"))
data

,label,name
0,1,00000.png
1,5,00001.png
2,1,00002.png
3,6,00003.png
4,3,00004.png
5,9,00005.png
6,7,00006.png
7,4,00007.png
8,5,00008.png
9,8,00009.png


In [ ]:
# 랜덤성 제어를 위한 seed 고정
import random
import numpy as np

seed = 0
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

# DataLoader

압축푼 직후에는 파일적용이 되지 않아 FileNotFoundError 오류가 뜰 수 있습니다.

그러한 경우 약간의 대기 시간 이후 다시 실행하면 됩니다.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms

from PIL import Image
from tqdm import tqdm
import pandas as pd
import json


In [ ]:
from glob import glob

class MyDataset(Dataset):
    def __init__(self, data_path, transform=None, train=True):
        self.train = train

        self.name2label = json.load(open(f"{data_path}/class_name.json"))


        if self.train:
            self.img_path = glob(f"{data_path}/train/*/*.JPEG")
            self.labels =  [self.name2label[d.split("/")[-2]] for d in self.img_path]
        else:
            self.img_path = glob(f"{data_path}/test/*.JPEG")

        self.transform = transform

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        if img.mode != 'RGB':
            img = img.convert('RGB')


        if self.transform:
            img = self.transform(img)

        if self.train:
            return img, self.labels[index]
        else:
            return img, self.img_path[index].split("/")[-1]


data_path = "./data/Kaggle02/"
transform =  transforms.Compose([
    transforms.ToTensor(),
])

train_data = MyDataset(data_path, train=True, transform=transform)
test_data = MyDataset(data_path, train=False, transform=transform)

# Split train data into train and validation
train_size = int(len(train_data) * 0.9)
train_data, val_data = torch.utils.data.random_split(train_data, [train_size, len(train_data) - train_size])


train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False)

In [ ]:
from matplotlib import pyplot as plt

name2label = json.load(open(f"{data_path}/class_name.json"))
label2name = {v:k for k, v in name2label.items()}

for idx, (images, labels) in enumerate(train_data):

    if idx > 20:
      break

    print(label2name[labels], idx)
    plt.imshow(images.permute(1, 2, 0))
    plt.show()


# Model

In [ ]:
from torchvision.models import resnet18
import torch
import torch.nn as nn

# Torchvision 라이브러리에서 모델 불러오기
model = resnet18(pretrained=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.fc = nn.Linear(512,10, bias=True)

model = model.to(device)

print(model)

In [ ]:
criterion = nn.CrossEntropyLoss() # 바꿔보기
optimizer = torch.optim.SGD(model.parameters()) # 바꿔보기

num_epochs = 5 # 바꿔보기
total_step = len(train_loader)

for epoch in range(num_epochs):
    total_loss = 0

    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    model.train()
    for i, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        pbar.set_description(f'Epoch [{epoch+1}/{num_epochs}], Loss: {round(total_loss / (i+1),4)}')


    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Accuracy of the model on the test images: {100 * correct / total} %')

Epoch [1/5], Loss: 2.2763: 100%|██████████| 36/36 [00:09<00:00,  3.63it/s]


Accuracy of the model on the test images: 13.0 %


Epoch [2/5], Loss: 2.171: 100%|██████████| 36/36 [00:06<00:00,  5.90it/s]


Accuracy of the model on the test images: 21.6 %


Epoch [3/5], Loss: 1.9326: 100%|██████████| 36/36 [00:06<00:00,  5.60it/s]


Accuracy of the model on the test images: 28.2 %


Epoch [4/5], Loss: 1.8175: 100%|██████████| 36/36 [00:06<00:00,  5.91it/s]


Accuracy of the model on the test images: 30.2 %


Epoch [5/5], Loss: 1.6632: 100%|██████████| 36/36 [00:06<00:00,  5.56it/s]


Accuracy of the model on the test images: 32.4 %


# Evaluation (Test)

In [ ]:
len(val_loader.dataset)

500

In [ ]:
correct = 0
total = len(val_loader.dataset)

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += torch.sum((predicted == labels)).item()

print(f'Accuracy : {100 * correct / total} %')

Accuracy : 32.4 %


# Make SubmitFile

In [ ]:
import pandas as pd

# 제출 파일 submission.csv 생성
outputs = {
    'Id': [],
    'Prediction': []
}

for images, id in tqdm(test_loader):
    model.eval()
    with torch.no_grad():
        output = model(images.to(device))
        _, predicted = torch.max(output, 1)
        outputs['Prediction'] += predicted.tolist()
        outputs['Id'] += id

df = pd.DataFrame(outputs)

df.to_csv('submission.csv', index=False, columns=['Id', 'Prediction'])

100%|██████████| 63/63 [00:08<00:00,  7.67it/s]


In [19]:
# 제출파일 다운로드
from google.colab import files

file_path = "submission.csv"
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>